In [2]:
import os
import pandas as pd 
import numpy as np 
from lyricsgenius import Genius
from dotenv import load_dotenv
import funkyFunctions
import re
import os
import pandas as pd 
import numpy as np 
from lyricsgenius import Genius
from dotenv import load_dotenv
import re
import requests


In [3]:
load_dotenv()  # Load environment variables from the .env file

genius_key = os.getenv('GENUIS_KEY')
genius = Genius(genius_key,timeout=15)
genius.verbose = False
genius.remove_section_headers = True
genius.excluded_terms = ["(Remix)", "(Live)"]

In [34]:
def getLove(rap_df):

    love_words = ['love', 'loves', 'loved', 'loving']
    song_number = 3
    artist = 3
    #Fill Songs and Lyrics Column with Nan
    rap_df['Songs'] = np.nan
    rap_df['Lyrics'] = np.nan
    #Show head
    print(rap_df.head())

    #Loop through rows
    #for x in range(loc(rap_df)):
    for x in range(artist):
        #Get artist name
        artist_name = rap_df.loc[x][0]
        print(f"Getting Data for {artist_name}")
        #Pull song data from genuis
        # Retry mechanism
        retries = 3
        while retries > 0:
            try:
                # Pull song data from genius
                artist = genius.search_artist(artist_name, max_songs=song_number, sort="popularity")
                break
            except requests.exceptions.Timeout:
                retries -= 1
                print(f"Request timed out. Retrying... {retries} attempts remaining.")
                if retries == 0:
                    print(f"Failed to get data for {artist_name}. Skipping...")
                    continue
        #Temp song and lyric list
        song_list = []
        song_lyrics_list = []

        #iiterate through songs and year 
        for song in artist.songs:
            print(song.title)
            song_list.append(song.title)
        # Add to row
        rap_df['Songs'][x] = song_list

        # Iterate over the songs again for lyrics
        for song in song_list:
            lyrics = genius.search_song(artist_name, song)
            
            # Split lyrics into lines
            lines = lyrics.lyrics.split('\n')
            
            # Iterate over the lines
            for line in lines:
                # Convert the line to lowercase for case-insensitive matching
                line_lower = line.lower()
                
                # If any love word is in the line, append it to song_lyrics_list
                if any(love_word in line_lower for love_word in love_words):
                    song_lyrics_list.append(line)

        # After the loops, update the DataFrame
        rap_df.loc[x, 'Songs'] = ', '.join(song_list)
        rap_df.loc[x, 'Lyrics'] = ', '.join(song_lyrics_list)

    print("Data Has Been Recived")
    #Show head
    print(rap_df.head())
    

In [35]:
csv_start = pd.read_csv('/home/lettuce/MyCode/Statistical-Analysis-of-Music-Genres/src/rap_ERA.csv')
csv_start.head()

,Rap Name,Era
0,The Notorious B.I.G.,1990s
1,Tupac,1990s
2,Eminem,2000s
3,Kendrick Lamar,2010s
4,Nas,1990s


In [36]:
getLove(csv_start)
#Save to new csv
csv_start.to_csv('/home/lettuce/MyCode/Statistical-Analysis-of-Music-Genres/src/rap_TRAN_LOVE.csv', index=False)
csv_start = pd.read_csv('/home/lettuce/MyCode/Statistical-Analysis-of-Music-Genres/src/rap_TRAN_LOVE.csv')
csv_start.head()

               Rap Name    Era  Songs  Lyrics
0  The Notorious B.I.G.  1990s    NaN     NaN
1                 Tupac  1990s    NaN     NaN
2                Eminem  2000s    NaN     NaN
3        Kendrick Lamar  2010s    NaN     NaN
4                   Nas  1990s    NaN     NaN
Getting Data for The Notorious B.I.G.
Juicy
Big Poppa
Suicidal Thoughts


/tmp/ipykernel_3468/3398968083.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rap_df['Songs'][x] = song_list


Getting Data for Tupac
Hit ’Em Up
Changes
Keep Ya Head Up
Getting Data for Eminem
Rap God
Killshot
Lose Yourself
Data Has Been Recived
               Rap Name    Era                                 Songs  \
0  The Notorious B.I.G.  1990s   Juicy, Big Poppa, Suicidal Thoughts   
1                 Tupac  1990s  Hit ’Em Up, Changes, Keep Ya Head Up   
2                Eminem  2000s      Rap God, Killshot, Lose Yourself   
3        Kendrick Lamar  2010s                                   NaN   
4                   Nas  1990s                                   NaN   

                                              Lyrics  
0  Funkmaster Flex, Lovebug Starski, Spread love,...  
1  With my AK, I'm still the thug that you love t...  
2  Everybody loves to root for a nuisance, That a...  
3                                                NaN  
4                                                NaN  


,Rap Name,Era,Songs,Lyrics
0,The Notorious B.I.G.,1990s,"Juicy, Big Poppa, Suicidal Thoughts","Funkmaster Flex, Lovebug Starski, Spread love,..."
1,Tupac,1990s,"Hit ’Em Up, Changes, Keep Ya Head Up","With my AK, I'm still the thug that you love t..."
2,Eminem,2000s,"Rap God, Killshot, Lose Yourself","Everybody loves to root for a nuisance, That a..."
3,Kendrick Lamar,2010s,NaN,NaN
4,Nas,1990s,NaN,NaN


In [38]:
csv_start['Lyrics'][0]


"Funkmaster Flex, Lovebug Starski, Spread love, it's the Brooklyn way, And she loves to show me off of course, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, I love it when you call me Big Poppa, She don't even love me like she did when I was younger (Yo, get a hold of yourself, nigga)"